<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment3_LinearModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
# Importing libraries that will be used 
import numpy as np
import tarfile
import glob
import re
import pandas as pd
#from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Loading the dataset

In [104]:
!wget http://www.cs.cornell.edu/people/pabo/movie-review-data/scale_data.tar.gz

--2021-10-14 04:17:12--  http://www.cs.cornell.edu/people/pabo/movie-review-data/scale_data.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4029756 (3.8M) [application/x-gzip]
Saving to: ‘scale_data.tar.gz.2’

scale_data.tar.gz.2 100%[===================>]   3.84M  --.-KB/s    in 0.1s    

2021-10-14 04:17:12 (29.9 MB/s) - ‘scale_data.tar.gz.2’ saved [4029756/4029756]



In [105]:
# Untar the dataset
my_tar = tarfile.open('/content/scale_data.tar.gz')
my_tar.extractall('/content/') 
my_tar.close()


In [106]:
id, review, rating, cl_3, cl_4 = [],[],[],[], []
for root in glob.glob ('/content/scaledata/*'):
  for path in glob.glob(root+'/*'): 
      fo = open(path)
      doc = fo.read()
      if 'id' in path:
        id.extend(doc.split('\n'))
      elif 'subj' in path:
          review.extend(doc.split('\n'))
      elif 'rating' in path:
        rating.extend(doc.split('\n'))
      elif '3class' in path:
        cl_3.extend(doc.split('\n'))
      else:
        cl_4.extend(doc.split('\n'))
      # print(path,'\n')
      # print(doc.split('\n')[:10])
print (len(id), len(review), len(rating), len(cl_3), len(cl_4))

df = pd.DataFrame(zip(id, review, rating, cl_3, cl_4), columns = ['id', 'review', 'rating', '3class', '4class'])
data = df[['review', '3class', '4class']].dropna()
data

5010 5010 5010 5010 5010


,review,3class,4class
0,i'm guessing -- and from the available evidenc...,0,0
1,"there's bad buzz , and then there's the the ba...",0,0
2,director : richard rush . director richard rus...,0,0
3,screenplay : johnny brennan & kamal ahmed and ...,0,0
4,screenplay : tim burns & tom stern and anthony...,0,0
...,...,...,...
5005,"since the film didn't have a director , the fi...",2,3
5006,claire denis ( i can't sleep/nenette and boni/...,2,3
5007,the young kid doesn't know what to say . yang-...,2,3
5008,the film's theme is about how women are made t...,2,3


# Pre-processing the data

In [107]:
data['3class'].value_counts()

1    1915
2    1894
0    1197
        4
Name: 3class, dtype: int64

In [108]:
data = data[data['3class']!='']


In [109]:
# Spliting the dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split (data['review'], data['3class'] , train_size = 0.8, random_state = 42, shuffle = True, stratify=data['3class'])
print ('Shapes of X_train, y_train: ', X_train.shape, y_train.shape)
print ('Shapes of X_test, y_test: ', X_test.shape, y_test.shape)

Shapes of X_train, y_train:  (4004,) (4004,)
Shapes of X_test, y_test:  (1002,) (1002,)


In [110]:
# Vectorizing the documents
vectorizer = CountVectorizer(binary = True)
X_train = vectorizer.fit_transform(X_train.to_list())
X_test = vectorizer.transform(X_test.to_list())
print ('Shapes of X_train, y_train: ', X_train.shape, y_train.shape)
print ('Shapes of X_test, y_test: ', X_test.shape, y_test.shape)

Shapes of X_train, y_train:  (4004, 38553) (4004,)
Shapes of X_test, y_test:  (1002, 38553) (1002,)


# Linear models

### Default

In [111]:
def printing_eval_scores (y_true, y_pred, report=''):
  accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
  precision = sklearn.metrics.precision_score(y_true, y_pred, average = 'weighted')
  recall = sklearn.metrics.recall_score(y_true, y_pred, average = 'weighted')
  f1 = sklearn.metrics.f1_score(y_true, y_pred, average = 'weighted')
  print('accuracy score: {:.3f}'.format(accuracy))
  print('precision score: {:.3f}'.format(precision))
  print('recall score: {:.3f}'.format(recall))
  print('F1 score: {:.3f}'.format(f1))
  if report is True:
    print(classification_report(y_true, y_pred))
  else:
    pass
  return accuracy, precision, recall, f1

In [112]:

def training (penalty, max_iter): #penalty="l2", max_iter=100
  SGD = SGDClassifier( penalty=penalty, max_iter=max_iter, shuffle=True).fit(X_train, y_train)
  y_pred = SGD.predict(X_test)
  printing_eval_scores (y_test, y_pred, report=True)

penalties = ['l1','l2']
for penalty in penalties:
  print('\n-------training linear model with penalty = {}'.format(penalty ))
  training (penalty='l2', max_iter=100)


-------training linear model with penalty = l1
accuracy score: 0.645
precision score: 0.646
recall score: 0.645
F1 score: 0.644
              precision    recall  f1-score   support

           0       0.65      0.55      0.60       240
           1       0.58      0.62      0.60       383
           2       0.71      0.73      0.72       379

    accuracy                           0.64      1002
   macro avg       0.65      0.63      0.64      1002
weighted avg       0.65      0.64      0.64      1002


-------training linear model with penalty = l2
accuracy score: 0.644
precision score: 0.646
recall score: 0.644
F1 score: 0.644
              precision    recall  f1-score   support

           0       0.61      0.57      0.59       240
           1       0.57      0.60      0.58       383
           2       0.75      0.73      0.74       379

    accuracy                           0.64      1002
   macro avg       0.64      0.63      0.64      1002
weighted avg       0.65      0.64  

Penalty = L2, and loss = log (logistic regression model) yeild the best performance on F1 score (0.657)

### Searching the best parameter

In [113]:
# Set the parameters by cross-validation
tuned_parameters = [{'alpha': [i for i in np.arange (0.0001, 0.01, 0.001 )]}] #, 'loss': ['log','hinge', 'perceptron']}]
score = 'f1'

print("# Searching hyper-parameters for %s" % score)
clf = GridSearchCV(SGDClassifier(shuffle=True), tuned_parameters, scoring='%s_macro' % score, cv = 5, refit = True, n_jobs = -1 )
clf.fit(X_train, y_train)
print('Best parameters set found on development set: {}'.format(clf.best_params_))
print("Grid scores on development set:")
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

y_pred =  clf.predict(X_test)
printing_eval_scores (y_test, y_pred, report=True)



# Searching hyper-parameters for f1
Best parameters set found on development set: {'alpha': 0.0001}
Grid scores on development set:
accuracy score: 0.632
precision score: 0.633
recall score: 0.632
F1 score: 0.632
              precision    recall  f1-score   support

           0       0.60      0.55      0.57       240
           1       0.56      0.59      0.58       383
           2       0.73      0.72      0.72       379

    accuracy                           0.63      1002
   macro avg       0.63      0.62      0.62      1002
weighted avg       0.63      0.63      0.63      1002



(0.6317365269461078,
 0.6327370182045454,
 0.6317365269461078,
 0.6318443469411966)

**(4)  SGDClassifier used “one versus all” (OVA) strategy to handle multi-class strategy. For each of K classes, the binary classifier will try to predict that class and the rest other K-1 classes.**

In [114]:
# Printing features with highest weights.
best_SGD = SGDClassifier( penalty='l2',  shuffle=True, alpha=0.0081).fit(X_train, y_train)
y_pred = best_SGD.predict(X_test)
weights = best_SGD.coef_
weights

array([[ 0.02311852, -0.04334723, -0.00866945, ...,  0.        ,
        -0.0067429 ,  0.        ],
       [ 0.02730253, -0.06957742, -0.02554108, ..., -0.01497236,
         0.00264218, -0.0132109 ],
       [-0.0636375 ,  0.10639395,  0.05170547, ...,  0.01590938,
         0.00497168,  0.        ]])

In [115]:
features = vectorizer.get_feature_names()

feature_weights = {}
for i in range(len(weights)):
  print('\n==============================================')
  print ('\nclass: {}'.format(i+1))
  for w in range(len(weights[i])):
     feature_weights[features[w]] = np.abs(weights[i][w])
  # Sorting the dictionary in descending order
  sorted_feature_weights = {k:v for k, v in sorted(feature_weights.items(), key = lambda item: item[1], reverse=True)}

  # Print the weights learned for each class
  print('10 most important features (in descending order): ')
  for k, v in list(sorted_feature_weights.items())[:20]:
    print ('{}: {:.5f}'. format(k,v))




class: 1
10 most important features (in descending order): 
boring: 0.26586
ridiculous: 0.25623
fine: 0.24660
satisfying: 0.24274
fans: 0.23504
flat: 0.22926
poorly: 0.22637
bad: 0.22252
awake: 0.22059
unfunny: 0.21288
watchable: 0.20999
effective: 0.20036
routine: 0.19843
fascinating: 0.19554
inane: 0.19362
problems: 0.19265
worst: 0.18591
entertaining: 0.18302
able: 0.18206
inspired: 0.18013


class: 2
10 most important features (in descending order): 
thumbs: 0.31618
add: 0.30385
problem: 0.28800
able: 0.27567
unfortunately: 0.26598
terrific: 0.25453
difficult: 0.25013
awake: 0.24836
sadly: 0.24220
frustrating: 0.24220
changes: 0.23868
masterpiece: 0.23163
albeit: 0.23075
choose: 0.23075
problems: 0.22635
promise: 0.22370
nice: 0.22282
though: 0.22018
interesting: 0.21842
dream: 0.21754


class: 3
10 most important features (in descending order): 
unfortunately: 0.29233
great: 0.28239
interesting: 0.27344
thumbs: 0.24660
perfect: 0.22870
flat: 0.22571
seriously: 0.22174
masterpiec

**Q6. How does the hyperplane of the classifier look like?**

In [116]:
import seaborn as sns
# sns.displot(weights)
hyperplane = best_SGD.decision_function(X_train).T
#sns.displot(np.flatnonzero(hyperplane))
hyperplane

array([[-1.10832476, -1.1035084 , -1.98971837, ..., -1.19501921,
        -1.72096556, -2.93950427],
       [ 1.0149452 ,  1.02287174, -1.05300128, ..., -1.01689148,
        -0.99135041, -1.05916637],
       [-1.0984601 , -1.44448901,  0.92203058, ...,  1.11095442,
         1.0552716 ,  1.10598274]])

In [117]:
for i in range(3):
  sns.displot(hyperplane[i], kind = 'hist')


**Q7. Model with log loss function**

In [118]:
log_SGD = SGDClassifier(loss='log', penalty='l2',  shuffle=True, alpha=0.0081).fit(X_train, y_train)
y_pred = log_SGD.predict(X_test)
printing_eval_scores (y_test, y_pred, report=True)


accuracy score: 0.672
precision score: 0.678
recall score: 0.672
F1 score: 0.672
              precision    recall  f1-score   support

           0       0.66      0.56      0.61       240
           1       0.59      0.68      0.63       383
           2       0.78      0.73      0.75       379

    accuracy                           0.67      1002
   macro avg       0.68      0.66      0.66      1002
weighted avg       0.68      0.67      0.67      1002



(0.6716566866267465,
 0.6783191784890963,
 0.6716566866267465,
 0.6724313473288569)

**8. Building a linear model to compare the result**

In [119]:
df = pd.DataFrame(zip(id, review, rating, cl_3, cl_4), columns = ['id', 'review', 'rating', '3class', '4class'])
new_df = df[['review', 'rating']].dropna()
new_df = new_df[new_df['rating'] != '']
new_df['rating'] =new_df['rating'].apply(lambda x: float(x))
     
   

In [120]:
new_df['rating'].describe()

count    5006.000000
mean        0.581422
std         0.181725
min         0.000000
25%         0.490000
50%         0.600000
75%         0.700000
max         1.000000
Name: rating, dtype: float64

In [121]:
plt.figure(figsize=(10,8))
sns.histplot(data = new_df, x = 'rating', bins = 20)
plt.show()


In [122]:
# Spliting the dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split (new_df['review'], new_df['rating'] , train_size = 0.8, random_state = 42, shuffle = True)
print ('Shapes of X_train, y_train: ', X_train.shape, y_train.shape)
print ('Shapes of X_test, y_test: ', X_test.shape, y_test.shape)

Shapes of X_train, y_train:  (4004,) (4004,)
Shapes of X_test, y_test:  (1002,) (1002,)


In [123]:
# Vectorizing the documents
vectorizer = CountVectorizer(binary = True)
X_train = vectorizer.fit_transform(X_train.to_list())
X_test = vectorizer.transform(X_test.to_list())
print ('Shapes of X_train, y_train: ', X_train.shape, y_train.shape)
print ('Shapes of X_test, y_test: ', X_test.shape, y_test.shape)

Shapes of X_train, y_train:  (4004, 38647) (4004,)
Shapes of X_test, y_test:  (1002, 38647) (1002,)


In [125]:
from sklearn.svm import LinearSVC
reg = LinearSVC().fit(X_train, y_train)
y_pred = reg.predict(X_test)
printing_eval_scores(y_test, y_pred)

ValueError: ignored